In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import scipy.integrate as integ
import ipywidgets as widgets

# Siły bezwładności

Poniższy przykład pokazuje siły odśrodkową i Coriolisa w działaniu. Dzieci kręcą się na karuzeli. W chwili początkowej, dziecko znajdujące się w dolnej części rysunku, rzuca piłkę w kierunku górnego dziecka. Trajektoria piłki pokazana jest na animacji, zarówno z perspektywy inercjalnego układu odniesienia (lewy rysunek), jak i z punktu widzenia układu nieinercjalnego (prawy rysunek).  Trajektoria w układzie nieinercjalnym została uzyskana poprzez rozwiązanie równania ruchu uwzględniającego siły pozorne

$$m\frac{d^2\vec{r}}{dt^2} = m \omega^2 \vec{r} - 2 m \vec{\omega}\times \vec{v}. $$

W rzeczywistości siły Coriolisa i dośrodkowa występują zawsze razem, ale ponieważ papier (i komputer) jest cierpliwy, to w symulacji możemy uwzględnić tylko jedną z nich. Sprawdź, która z tych sił ma większy wpływ (i jaki) na trajektorię piłki. Jakie znaczenie ma tu prędkość, z jaką obraca się karuzela?

In [2]:
def fictitious(y, t, omega, coriolis = True, centrifugal = True):
    x, vx, y, vy = y
    
    r = np.array([x, y, 0])
    v = np.array([vx, vy, 0])
    w = np.array([0, 0, omega])
    
    a = 0
    if coriolis:
        a -= 2 * np.cross(w, v)
    if centrifugal:
        a += omega ** 2 * r 
    
    return vx, a[0], vy, a[1]

def fictitious_coriolis(y, t, omega):
    x, vx, y, vy = y
    
    r = np.array([x, y, 0])
    v = np.array([vx, vy, 0])
    w = np.array([0, 0, omega])
    
    a = - 2 * np.cross(w, v)
    
    return a[0], a[1]

def fictitious_centrifugal(y, t, omega):
    x, vx, y, vy = y
    
    r = np.array([x, y, 0])
    v = np.array([vx, vy, 0])
    w = np.array([0, 0, omega])
    
    a = omega ** 2 * r
    
    return a[0], a[1]

In [3]:
omega = 1
t = np.linspace(0, 2, 2000)
v0 = 2

arrow_scale = 0.18

def draw_frame(index, initial_angle, show_velocity, show_coriolis, show_centrifugal, show_radial, simulation, show_real_trajectory, show_coriolis_trajectory, show_centrifugal_trajectory, show_noforce_trajectory, show_shadows):
    v0x = v0 * np.sin(np.radians(initial_angle))
    v0y = v0 * np.cos(np.radians(initial_angle))
    fict = integ.odeint(fictitious, (0, v0x, -1, v0y), t, args=(1,))
    fict_coriolis = integ.odeint(fictitious, (0, v0x, -1, v0y), t, args=(1, True, False))
    fict_centrifugal = integ.odeint(fictitious, (0, v0x, -1, v0y), t, args=(1, False, True))    
    if simulation == 'everything':
        x, vx, y, vy = fict.T
    elif simulation == 'coriolis only':
        x, vx, y, vy = fict_coriolis.T
    elif simulation == 'centrifugal only':
        x, vx, y, vy = fict_centrifugal.T        
        
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(20, 10)

    ax1.set_aspect(1)
    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    car = plt.Circle((0, 0), 1.2, color = "black", fill = False, lw = 2, zorder = 2)
    ax1.add_artist(car)
    rupper = np.array([-np.sin(omega * t[index]), np.cos(omega * t[index])])
    upper = plt.Circle(rupper, 0.1, color = "C0")
    ax1.add_artist(upper)
    rlower = -rupper
    upper = plt.Circle(rlower, 0.1, color = "C1")
    ax1.add_artist(upper)
    ax1.plot((omega + v0x) * t[0:index], v0y * t[0:index] - 1, "C2", lw = 3)
    ball = plt.Circle(((omega + v0x) * t[index], v0y * t[index] - 1), 0.05, color = "C3", zorder = 3)
    ax1.add_artist(ball)
    
    if show_velocity:
        arrow_velocity = plt.Arrow((omega + v0x) * t[index], v0y * t[index] - 1, arrow_scale * (omega + v0x), arrow_scale * v0y, color = 'C2', width = 0.1, zorder = 2)
        ax1.add_artist(arrow_velocity)
    
    ax2.set_aspect(1)
    ax2.set_xlim(-1.5, 1.5)
    ax2.set_ylim(-1.5, 1.5)
    car = plt.Circle((0, 0), 1.2, color = "black", fill = False, lw = 2, zorder = 2)
    ax2.add_artist(car)
    rupper = np.array([0, 1])
    upper = plt.Circle(rupper, 0.1, color = "C0")
    ax2.add_artist(upper)
    rlower = -rupper
    upper = plt.Circle(rlower, 0.1, color = "C1")
    ax2.add_artist(upper)
    ax2.plot(x[0:index], y[0:index], "C2", lw = 3)
    ball = plt.Circle((x[index], y[index]), 0.05, color = "C3", zorder = 3)
    ax2.add_artist(ball)
    
    if show_real_trajectory:
        xr, _, yr, _ = fict.T
        ax2.plot(xr[0:index], yr[0:index], "C2", lw = 3, linestyle = '--')
        if show_shadows:
            ball = plt.Circle((xr[index], yr[index]), 0.05, color = "C3", zorder = 2, alpha = 0.25)
            ax2.add_artist(ball)
    if show_noforce_trajectory:
        ax2.plot(v0x * t[0:index], v0y * t[0:index] - 1, "C2", lw = 3, linestyle = '--')
        if show_shadows:
            ball = plt.Circle((v0x * t[index], v0y * t[index] - 1), 0.05, color = "C3", zorder = 2, alpha = 0.25)
            ax2.add_artist(ball)
    if show_coriolis_trajectory:
        xc, _, yc, _ = fict_coriolis.T
        ax2.plot(xc[0:index], yc[0:index], "C2", lw = 3, linestyle = '--')
        if show_shadows:
            ball = plt.Circle((xc[index], yc[index]), 0.05, color = "C3", zorder = 2, alpha = 0.25)
            ax2.add_artist(ball)
    if show_centrifugal_trajectory:
        xc, _, yc, _ = fict_centrifugal.T
        ax2.plot(xc[0:index], yc[0:index], "C2", lw = 3, linestyle = '--')
        if show_shadows:
            ball = plt.Circle((xc[index], yc[index]), 0.05, color = "C3", zorder = 2, alpha = 0.25)
            ax2.add_artist(ball)            
    
    
    if show_coriolis:
        coriolis_x, coriolis_y = fictitious_coriolis((x[index], vx[index], y[index], vy[index]), t[index], omega)
        arrow_coriolis = plt.Arrow(x[index], y[index], arrow_scale * coriolis_x, arrow_scale * coriolis_y, color = 'C9', width = 0.1, zorder = 2)
        ax2.add_artist(arrow_coriolis)
    if show_velocity:
        arrow_velocity = plt.Arrow(x[index], y[index], arrow_scale * vx[index], arrow_scale * vy[index], color = 'C2', width = 0.1, zorder = 2)
        ax2.add_artist(arrow_velocity)
    if show_centrifugal:
        centrifugal_x, centrifugal_y = fictitious_centrifugal((x[index], vx[index], y[index], vy[index]), t[index], omega)        
        arrow_centrifugal = plt.Arrow(x[index], y[index], arrow_scale * centrifugal_x, arrow_scale * centrifugal_y, color = 'C5', width = 0.1, zorder = 2)
        ax2.add_artist(arrow_centrifugal)
    if show_radial:
        line_centrifugal = plt.Line2D((0, x[index]), (0, y[index]), linestyle = '--', color = 'C5', zorder = 1, linewidth = 2)
        ax2.add_artist(line_centrifugal)
    
widgets.interact(draw_frame, index = widgets.Play(value = 0, min = 0, max = np.size(t) - 1, step = 1, interval = 10),
                             initial_angle = widgets.IntSlider(value = 0, min = -90, max = 90, step = 1, description = 'initial velocity angle'),
                             show_velocity = widgets.Checkbox(value = False, description = 'show velocity vector'),
                             show_coriolis = widgets.Checkbox(value = False, description = 'show Coriolis force'),
                             show_centrifugal = widgets.Checkbox(value = False, description = 'show centrifugal force'),
                             show_radial = widgets.Checkbox(value = False, description = 'show radial direction'),
                             simulation = widgets.Dropdown(options = ('everything', 'coriolis only', 'centrifugal only'), index = 0),
                             show_real_trajectory = widgets.Checkbox(value = False, description = 'show real trajectory'),
                             show_coriolis_trajectory = widgets.Checkbox(value = False, description = 'show coriolis trajectory'),
                             show_centrifugal_trajectory = widgets.Checkbox(value = False, description = 'show centrifugal trajectory'),
                             show_noforce_trajectory = widgets.Checkbox(value = False, description = 'show no-force trajectory'),
                             show_shadows = widgets.Checkbox(value = True, description = 'show ball trajectory shadow'))

interactive(children=(Play(value=0, description='index', interval=10, max=1999), IntSlider(value=0, descriptio…

<function __main__.draw_frame(index, initial_angle, show_velocity, show_coriolis, show_centrifugal, show_radial, simulation, show_real_trajectory, show_coriolis_trajectory, show_centrifugal_trajectory, show_noforce_trajectory, show_shadows)>